In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Importing required libraries
import os
from IPython.display import Image
from IPython.core.display import HTML

**To analyze a gravitational wave, we need to provide a prior, a model, and data settings in our configuration file. These settings can all be put into a single file or split up into multiple files, all of which are provided via the "--config-file" option. We split up the configuration file upto 3 files:**

**gw150914_like.ini -> Specifices the model we will use, and the parameters that will be varied and their priors. This is the configuration file you would use to analyze signals like GW150914.**

**emcee_pt-gw150914_like.ini -> Specifies sampler settings. In this case, we are using the emcee_pt sampler.**

**data.ini -> specifies the settings for lodaing the gravitational-wave data.**

# Determining analysis time duration

**Why did we use -8 and +2 for the analysis times? <br> The discrete inner product treats the data as if it were cyclic. If we try to filter a model waveform that is longer (starting from the low-frequency-cutoff) than the analysis duration, it will wrap around to the beginning.<br>For example, if we used an analysis time of 4s, but a waveform is 5s long, the last second of the waveform will wrap around to lay on top of the first second of the segment. To avoid this, we need to analyze a segment that is longer than the longest possible waveform admitted by our prior plus our uncertainty in the trigger time (
s). In this case, our longest waveform is 
. We can check the duration of this waveform using get_waveform_filter_length_in_time:**

In [3]:
from pycbc import waveform

waveform.get_waveform_filter_length_in_time(approximant='IMRPhenomPv2', mass1=10., mass2=10.,
                                            spin1z=0.99, spin2z=0.99, f_lower=20.)

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


array(7.14267385)

**Since the trigger-time is near the merger time, using ~8s before and ~2s after sufficiently encompasses the longest waveform we might sample.**

# Run it

**Before we can run, we need to download frame files from GWOSC. These contain the LIGO data that we will analyze.**

In [7]:
if not os.path.exists('/home/nperla/Gravitational_Waves/code/Inference/Inference_4_Analyzing_a_gravitational_wave/H-H1_GWOSC_4KHZ_R1-1126257415-4096.gwf'):
    !wget https://gwosc.org/eventapi/html/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_16KHZ_R1-1126259447-32.gwf
if not os.path.exists('/home/nperla/Gravitational_Waves/code/Inference/Inference_4_Analyzing_a_gravitational_wave/L-L1_GWOSC_4KHZ_R1-1126257415-4096.gwf'):
    !wget https://gwosc.org/eventapi/html/GWTC-1-confident/GW150914/v3/L-L1_GWOSC_16KHZ_R1-1126259447-32.gwf

--2023-04-23 18:28:02--  https://gwosc.org/eventapi/html/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_16KHZ_R1-1126259447-32.gwf
Resolving gwosc.org (gwosc.org)... 131.215.113.73
Connecting to gwosc.org (gwosc.org)|131.215.113.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4050875 (3.9M) [application/octet-stream]
Saving to: ‘H-H1_GWOSC_16KHZ_R1-1126259447-32.gwf.1’

H-H1_GWOSC_16KHZ_R1 100%[===================>]   3.86M  2.04MB/s    in 1.9s    

2023-04-23 18:28:05 (2.04 MB/s) - ‘H-H1_GWOSC_16KHZ_R1-1126259447-32.gwf.1’ saved [4050875/4050875]

--2023-04-23 18:28:05--  https://gwosc.org/eventapi/html/GWTC-1-confident/GW150914/v3/L-L1_GWOSC_16KHZ_R1-1126259447-32.gwf
Resolving gwosc.org (gwosc.org)... 131.215.113.73
Connecting to gwosc.org (gwosc.org)|131.215.113.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3900380 (3.7M) [application/octet-stream]
Saving to: ‘L-L1_GWOSC_16KHZ_R1-1126259447-32.gwf.1’

L-L1_GWOSC_16KHZ_R1 100%[===

**To run this example, we will just run the bash script. Running this example to completion will take several hours. Instead, we'll just run for a couple of checkpoints, kill it, then start it again to see how checkpointing works.**

**First, do the following:**

<li>Set the checkpoint interval in the config file to 4.</li>

**Now run the script. Watch the output. After it checkpoints (it will say "Running sampler for 4 to 8 iterations"), interrupt the kernel by hitting the stop button above.**

In [5]:
!bash run.sh

2023-04-23T18:27:00.795-04:00 Using seed 1897234
2023-04-23T18:27:00.796-04:00 Running with CPU support: 1 threads
2023-04-23T18:27:00.867-04:00 Reading configuration file
2023-04-23T18:27:00.870-04:00 Setting up model
2023-04-23T18:27:00.872-04:00 Setting up priors for each parameter
2023-04-23T18:27:00.874-04:00 Sampling in mchirp, q in place of mass1, mass2
2023-04-23T18:27:00.874-04:00 Loading waveform transforms
2023-04-23T18:27:00.878-04:00 Determining analysis times to use
2023-04-23T18:27:00.878-04:00 Padding H1 analysis start and end times by 4 (= psd-inverse-length/2) seconds to account for PSD wrap around effects.
2023-04-23T18:27:00.878-04:00 Padding L1 analysis start and end times by 4 (= psd-inverse-length/2) seconds to account for PSD wrap around effects.
usage: pycbc_inference [-h] --instruments INSTRUMENTS [INSTRUMENTS ...]
                       [--trigger-time TRIGGER_TIME] --analysis-start-time
                       IFO:TIME [IFO:TIME ...] --analysis-end-time IFO:T